#### The Python notebook with the Data Cleaning, Model initailization, & Model fine-tuning code

### - Install Needed Libraries

In [1]:
! pip install -U accelerate --quiet
! pip install --upgrade accelerate
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git
! git clone https://github.com/aub-mind/arabert.git
! pip install evaluate
! pip install gradio

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-uy3toi3p
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-uy3toi3p
  Resolved https://github.com/huggingface/transformers.git to commit abaca9f9432a84cfaa95531de4c72334f38a42f2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3-none-any.whl size=7286950 sha256=6cd53f7dd1cacf9d88651abc7c4e83bf92ebbe6f5afa644fc6656a6df35e39a8
  Stored in directory: /tmp/pip-ephem-wheel-cache-wsh4d925/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.1
    Uninstalling transformers-4.30.1:
      Successfully uninstalled transformers-4.30.1
  Cloning https://github.com

### Import Needed libraries

In [2]:
import pandas as pd
import numpy as np
import re
from arabert.aragpt2.grover.modeling_gpt2 import GPT2LMHeadModel
from torch.utils.data import Dataset, DataLoader
import torch
import evaluate
from evaluate import logging
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
from transformers import AutoTokenizer
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import GPT2TokenizerFast, pipeline
from transformers import  AutoModel, utils
# from transformers import BertTokenizer,BertForQuestionAnswering
from transformers import  Trainer, TrainingArguments,AutoModelWithLMHead, GPT2LMHeadModel

### - Read Data 

In [4]:
df = pd.read_csv('/kaggle/input/fatawa/88kData.csv')
df.head(10)
# No. of records
len(df)

88606

### - Cleaning data

In [5]:
#remove ID column
df = df.drop('ID', axis=1)
# remove Null value
df = df.dropna()

In [6]:
NewList = []

In [7]:
# append unneeded fatwa

NewList+=list(df[df['ans'].str.contains('انظر إلى الفتوى')].index)
NewList+=list(df[df['ans'].str.contains('رقم:')].index)
NewList+=list(df[df['ans'].str.contains('فتوى رقم')].index)

In [8]:
# drop unneeded fatwa
df = df.drop(index=NewList, axis=0)

In [9]:
df[df['ans'].str.contains('นู')]

,title,ques,ans
84488,ู ุฐุงูุจ ุงูุนูู ุงุก ูู ุงูุฃุญู ุจุง...,ู ุงุชุช ุฃู ู ููุง ุฃุฐูุฑ ุฅู ูุงูุช ...,ุงูุญู ุฏ ููู ูุงูุตูุงุฉ ูุงูุณูุงู...


In [10]:
df = df.drop(index=84488, axis=0)

In [ ]:
# No. of new records
len(df)

In [11]:
def extract_arabic_and_english_numerics(text):
    # remove english letter
    arabic_pattern = r'[^a-zA-Z\s0-9]+'
    numeric_pattern = r'\d+'
    combined_pattern = f'({arabic_pattern}|{numeric_pattern})'
    
    matches = re.findall(combined_pattern, text)    
    extracted_text = ' '.join(matches)
    
    return extracted_text

In [12]:
#Applying the function on the ['ans'] column
df['ans'] = df['ans'].apply(extract_arabic_and_english_numerics)
df['ans']

0        الحمد لله والصلاة والسلام على رسول الله وعلى آ...
1        الحمد لله والصلاة والسلام على رسول الله وآله و...
2        الحمد لله والصلاة والسلام على رسول الله وعلى آ...
3        الحمد لله والصلاة والسلام على رسول الله وآله و...
4        الحمد لله والصلاة والسلام على رسول الله وآله و...
                               ...                        
88588    الحمد لله والصلاة والسلام على رسول الله وعلى آ...
88590    الحمد لله والصلاة والسلام على رسول الله وعلى آ...
88596    الحمد لله والصلاة والسلام على رسول الله وعلى آ...
88597    الحمد لله والصلاة والسلام على رسول الله وعلى آ...
88604    الحمد لله والصلاة والسلام على رسول الله وعلى آ...
Name: ans, Length: 43849, dtype: object

In [13]:
#Applying the function on the ['ques'] column
df['ques'] = df['ques'].apply(extract_arabic_and_english_numerics)
df['ques']

0        ما حكم شراء السيارة بالأقساط المريحة من بنك رب...
1        امرأة توفيت ولها مبلغ من المال ، ولها زوج طاعن...
2             ما حكم الإجازات المرضية فى حالة مرض أولادي ؟
3        ما هو سجود السهو ، كيفيته، ووقته؟ وما حكم من ش...
4        هل يمكن للأخت أن ترضع أخاها، في حال مرض الأم؟ ...
                               ...                        
88588    أمي مصابة بقصور الكلى وهي بذلك مجبرة على التصف...
88590    سؤالي هو: إذا كان في ذمة الشخص دين (بضاعة) ولم...
88596    هل يصح للإمام التنكيس في الصلاة بمعنى أن يقرأ ...
88597    أنا متزوجة من 12 سنة، وأعاني من أن زوجي إذا زا...
88604    أنا أعمل في شركة ما، وعندي شقة فى مدينة من الم...
Name: ques, Length: 43849, dtype: object

In [14]:
#Applying the function on the ['title'] column
df['title'] = df['title'].apply(extract_arabic_and_english_numerics)
df['title']

0                       الشراء عن طريق البنوك الربوية حرام
1          لابد من حصر الورثة لتتم قسمة التركة بصورة شرعية
2           لا ينبغي للموظف أن يخالف الشرع ليحصل على إجازة
3                                         أحكام سجود السهو
4                      إذا أرضعت الأخت أخاها صار ابناً لها
                               ...                        
88588     هل رفض الأم تبرع ولدها لها بكليته يعد قتلا للنفس
88590                      من يئس من إيصال الأمانة لصاحبها
88596           حكم عدم مراعاة ترتيب السور في قراءة الصلاة
88597    تضييق الزوج على أهل زوجته من زيارتها ليس من ال...
88604                   ارجع إلى شركتك وأعلمها بما هو كائن
Name: title, Length: 43849, dtype: object

In [15]:
newlist2 = []

In [16]:
#Visualize the answers with length less than 118 charachter
for k in range(len(df)):
    
    if len(df['ans'].iloc[k]) < 118:
        print(df['ans'].iloc[k])

, ; . , , - , . . , , , . . ' . . .
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه أما بعد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه أما بعد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه أما بعد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد: فقد سبق بيان حكمه بالفتوى برقم 7051 .
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد: 12 - جوابه في الذي قبله.
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله 

In [17]:
#Dropping every row that contains an answer with length less than 118 charachter
for k in range(len(df)):
    
    if len(df['ans'].iloc[k]) < 118:
        newlist2.append(k)

In [18]:
# remove from data
newlist2 = list(df.iloc[newlist2].index)
df = df.drop(index=newlist2, axis=0)

In [19]:
#Making all the dataset columns as a string, and dropping duplicates from the dataset.
df = df.astype(str)
df.drop_duplicates(inplace=True)

In [20]:
# resize data to 20000
df = df[0:20000]

In [21]:
#Save the cleaned version of the data to the Machine as .csv file
df.to_csv("Finalized_Data2.csv",index=False)

In [22]:
df = pd.read_csv("Finalized_Data2.csv")

### - Model Intialization & Data Adjustement for Model Fine-Tuning

In [23]:
df["full_text"]=" question: " + df["title"]+df["ques"]+" answer: "+df["ans"]

In [24]:
df["full_text"][1]

' question: لابد من حصر الورثة لتتم قسمة التركة بصورة شرعيةامرأة توفيت ولها مبلغ من المال ، ولها زوج طاعن في السن ، ولها أيضاً أولاد. فكيف يتم توزيع المبلغ .. أم أن الزوج أحق بهذا المال ؟ answer: الحمد لله والصلاة والسلام على رسول الله وآله وصحبه أما بعد: فليس الزوج أحق بهذا المال، بل يجب أن يقسم مع تركتها كما أمر الله جل وعلا وبين رسوله صلى الله عليه وسلم، وحيث توفيت المرأة وتركت زوجا وأولادا فتوزع التركة بينهم للزوج الربع فرضا والباقي للأولاد تعصيبا للذكر مثل حظ الأنثيين، والأولى أن تراجع في ذلك المحكمة الشرعية في بلده إذا وجدت لإعلان الوراثة فربما يكون ثم وارث لم يذكره السائل أو لا يعلم أنه يرث. وبالله التوفيق.'

In [25]:
# split the data to train and test

from sklearn.model_selection import train_test_split
train, test = train_test_split(df["full_text"],test_size=0.10, shuffle=True, random_state=42)

In [26]:
# save train and test data 

train.to_csv(r'train_dataset1.txt', header=None, index=None, sep=' ', mode='a')
test.to_csv(r'test_dataset1.txt', header=None, index=None, sep=' ', mode='a')

### Pretrain aragpt2 model

In [27]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 5.7 MB/s eta 0:00:00


In [28]:
from sacrebleu import corpus_bleu
import numpy as np

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

    # Set pad token
    if tokenizer.pad_token_id is not None:
        labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    else:
        labels_ids[labels_ids == -100] = 0

    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    bleu_score = round(corpus_bleu(pred_str, [label_str]).score, 4)

    return {"bleu": bleu_score}


In [29]:
model = AutoModelWithLMHead.from_pretrained("aubmindlab/aragpt2-base")
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-base")
tokenizer.add_tokens(['<question>', '<answer>'])
train_path = 'train_dataset1.txt'
test_path = 'test_dataset1.txt'

In [30]:
model.config.max_length=100

### - (ARAGPT2) Fine-Tuning

In [31]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_path,
        block_size=128
    )

    test_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=test_path,
        block_size=128
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False
    )
    
    eval_data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False
    )

    return train_dataset, test_dataset, data_collator, eval_data_collator

train_dataset, test_dataset, data_collator, eval_data_collator = load_dataset(train_path, test_path, tokenizer)


In [32]:
eval_data_collator

DataCollatorForLanguageModeling(tokenizer=GPT2TokenizerFast(name_or_path='aubmindlab/aragpt2-base', vocab_size=64000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True), mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

In [33]:
len(train_dataset)

61800

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [35]:
# clean memory
torch.cuda.empty_cache()

In [36]:
batch_size=8

In [37]:
# !pip install --upgrade transformers

In [38]:
from transformers import  Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer

In [39]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./bert2gpt-Training",
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    num_train_epochs=2,
    do_train=True,
    do_eval=True,
    fp16=True,
    overwrite_output_dir=True,
    learning_rate = 1e-5,
    weight_decay=0.01,
    warmup_ratio = 0.05,
    seed = 1995,
    save_total_limit = 2,
    load_best_model_at_end = True,
    #push_to_hub=True,
    optim = "adamw_hf",
)
#

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator = data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset

)
#    save_total_limit=3,
#

In [40]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss



KeyboardInterrupt



In [ ]:
print(trainer.state.log_history)

In [ ]:
import matplotlib.pyplot as plt

logs = trainer.state.log_history

train_loss_values = [log.get('loss') for log in logs]
validation_loss_values = [log.get('eval_loss') for log in logs]
epochs = [log.get('epoch') for log in logs]

# Plotting the train loss and validation loss
plt.plot(epochs, train_loss_values, label='Train Loss')
plt.plot(epochs, validation_loss_values, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

train_loss_values = [log["train_loss"] for log in trainer.state.log_history[1:]]
validation_loss_values = [log["eval_loss"] for log in trainer.state.log_history[0:]]

epochs = range(1, len(train_loss_values) + 1)

plt.plot(epochs, train_loss_values, label='Train Loss')
plt.plot(epochs, validation_loss_values, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("aubmindlab/aragpt2-base")
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-base")

In [ ]:
# حساب المقاييس على مجموعة الاختبار
eval_results = trainer.evaluate()

# عرض نتائج المقاييس
print("Meteor Score:", eval_results['eval_meteor_score'])
print("ROUGE2 Precision:", eval_results['eval_rouge2_precision'])
print("ROUGE2 Recall:", eval_results['eval_rouge2_recall'])
print("ROUGE2 F-Measure:", eval_results['eval_rouge2_f_measure'])

In [ ]:
import gradio as gr
from transformers import  pipeline

generation_pipeline = pipeline('text-generation',model="/kaggle/input/islamweb", tokenizer='aubmindlab/aragpt2-base')


def prediction(str):

    str=generation_pipeline(str,max_length=300)[0]['generated_text']
    start_index = str.index('answer:')
    if 'والله أعلم' in str:
        stop_index = str.index('والله أعلم')
        str = str[start_index+8:stop_index+11].strip()
        
    else:
        str = str[start_index+8:].strip()

    return str

demo = gr.Interface(fn=prediction,outputs=gr.Textbox(label="الاجابه"),inputs=gr.Textbox(label="ما هو سؤالك؟"),title="نَوَّرْ",css="body {background-color: green,}")

demo.launch(share=True)

In [ ]:
trainer.save_model()
trainer.save_state()